In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [4]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.data = torch.FloatTensor(x)
        self.lable = torch.LongTensor(y)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return (self.data[idx], self.lable[idx])

X_train = np.random.randn(10000, 20)
Y_train = np.zeros(10000,)#((np.sum(X_train[:, :10], axis=1))>0).astype(int)
X_dev = np.random.randn(1000, 20)
Y_dev = np.zeros(1000,)#((np.sum(X_dev, axis=1))>0).astype(int)
print(Y_train[:10])
print(Y_dev[:10])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
dataset_train = MyDataset(X_train, Y_train)
dataset_dev = MyDataset(X_dev, Y_dev)
dataloader_train = DataLoader(dataset_train, batch_size=20)

dataloader_dev = DataLoader(dataset_dev, batch_size=20,
                        shuffle=True, num_workers=4)

iter_train = iter(dataloader_train)

In [ ]:


class MyModel(nn.Module):
    def __init__(self, input_dim, num_class):
        super(MyModel, self).__init__()
        self.network = nn.Sequential(
                       nn.Linear(input_dim, input_dim),
                       nn.Dropout(p=0.5),
                       nn.ReLU(),
                       nn.Linear(input_dim, input_dim),
                       nn.ReLU(),
                       nn.Linear(int(input_dim), num_class)
                        )
    def forward(self, X):
        out = self.network(X)
        return out
        
model = MyModel(20, 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epoch = 10
for epoch in range(num_epoch):
    model.train()
    for (x, y) in dataloader_train:
        optimizer.zero_grad()
        logits = model(x)
        loss = F.cross_entropy(logits, y)
        loss.backward()
        optimizer.step()
    
    model.eval()
    tot = 0
    correct = 0
    for (x, y) in dataloader_dev:
        logits = model(x)
        predict = logits.argmax(dim=1)
        tot += x.size(0)
        correct += (predict == y).float().sum()
    acc = correct / tot
    print(f'epoch:{epoch} acc：{acc}')